In [6]:
import pandas as pd
import os
from collections import Counter
# Import the Training Dataset
df = pd.read_csv('train_revised.csv')
df.head(2)

,ride_id,seat_number,payment_method,payment_receipt,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity
0,1442,15A,Mpesa,UZUEHCBUSO,17-10-17,7:15,Migori,Nairobi,Bus,49
1,5437,14A,Mpesa,TIHLBUSGTE,19-11-17,7:12,Migori,Nairobi,Bus,49


In [13]:
def mode(x):
    '''Returns the most repeated element from a list'''
    F = Counter(x)
    F = F.most_common(1)[0][0]# the counter object returns a tuple,we want the first element of the first tuple
    
    return F

Df = df.groupby('ride_id')
Y = Df.ride_id.agg('count') # The target variable is how many tickets were sold

# Generate the X dataset from the groupby object
columns = []
Df = Df.agg(mode) # Create a dataset using the Mode function that we created

columns = ['payment_method','travel_date','travel_from','travel_to','car_type','max_capacity']

X = Df[columns]# slice the dataset using the columns that we had created.
print(X.shape,Y.shape)

# Create Some New columns using the Dates

X.travel_date = pd.DatetimeIndex(X.travel_date) # Convert the date into a datetime index
X['Day'] = [i.day for i in X.travel_date] # Day of the month
X['Day_of_week'] = [i.dayofweek for i in X.travel_date ] # Which day of the week is it, where sunday is 0 and saturday is 6
X['is_weekend'] = [1 if (i == 5 or i == 6) else 0 for i in X['Day_of_week']] # returns 1 if the day is weekend 0 otherwise
X['Month'] = [i.month for i in X.travel_date] # January is 1, up to december 12
X['Year'] = [i.year for i in X.travel_date] # either 2017 or 2018
# Create Dummies For the cols
Categorical_variables = ['travel_from','car_type','Year','payment_method']
Numerical_variables = ['Day_of_week','Month','is_weekend']
X_clean = X[Numerical_variables]
for i in Categorical_variables:# convert the Categorical columns into numerical columns
    col = X[i]
    Dummy = pd.get_dummies(col)
    X_final = pd.concat((X_clean,Dummy),1) # add the dummies to the original numerical dataset
    X_clean = X_final # return a final dataset
print(X_final.shape,Y.shape)

X_final['Y'] = Y # You can add back the Target Variable to the predictor dataset

(6249, 6) (6249,)


/home/kamau/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/kamau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/kamau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

(6249, 26) (6249,)


In [177]:
'''# Test the Correlations
#X_fin.corr(method  = 'pearson')
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]
X_fin = calculate_vif_(X_fin)
X_fin.shape
'''

"# Test the Correlations\n#X_fin.corr(method  = 'pearson')\nfrom statsmodels.stats.outliers_influence import variance_inflation_factor    \n\ndef calculate_vif_(X, thresh=5.0):\n    variables = list(range(X.shape[1]))\n    dropped = True\n    while dropped:\n        dropped = False\n        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)\n               for ix in range(X.iloc[:, variables].shape[1])]\n\n        maxloc = vif.index(max(vif))\n        if max(vif) > thresh:\n            print('dropping '' + X.iloc[:, variables].columns[maxloc] +\n                  '' at index: ' + str(maxloc))\n            del variables[maxloc]\n            dropped = True\n\n    print('Remaining variables:')\n    print(X.columns[variables])\n    return X.iloc[:, variables]\nX_fin = calculate_vif_(X_fin)\nX_fin.shape\n"

## Baseline Model Without Uber Data

In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as scale


x_train,x_test,y_train,y_test = train_test_split(X_final,Y,test_size = 0.1)

Scaled = scale() # Scale the data to be between 1 and 0 for all the predictor columns
Scaled.fit(x_train) # fit the data for scaling
Clean_transformed_data= Scaled.transform(x_train) # transform the fitted data into the correct format. 

RF = RandomForestRegressor(verbose = 1,n_estimators = 100)# train the regression model,verbose means it will narrate what the model is doing, n_estimators =  how many trees the model will train
RF.fit(Clean_transformed_data,y_train)
print(Clean_transformed_data.shape)


Pred = RF.predict(x_test)

print('The Mean Absolute Value for the Models is:',(mae(y_test,Pred))) # prints the mean_absolute_value for the model

/home/kamau/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/kamau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


(5624, 27)
The Mean Absolute Value for the Models is: 29.628944


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


## Testing The Model on the Test Set

In [25]:
test =  pd.read_csv('test_questions.csv')
X = test

# Create a similar dataset to the one we used for the training
X.travel_date = pd.DatetimeIndex(X.travel_date)
X['Day'] = [i.day for i in X.travel_date]
X['Day_of_week'] = [i.dayofweek for i in X.travel_date ]
X['is_weekend'] = [True if (i == 5 or i == 6) else False for i in X['Day_of_week']]
X['Month'] = [i.month for i in X.travel_date]
X['Year'] = [i.year for i in X.travel_date]

Categorical_variables = ['travel_from','car_type','Year']
Numerical_variables = ['Day_of_week','Month','is_weekend']
X_test_clean = X[Numerical_variables]
for i in Categorical_variables:
    col = X[i]
    Dum = pd.get_dummies(col)
    X_fins = pd.concat((X_test_clean,Dum),1)
    X_test_clean = X_fins
print(X_fins.shape,X_final.shape)
#Check if there are some columns that were in the training set that are not in the testing set
not_in_the_test_data = []   # empty list for this purpose  
for i in X_final.columns:
    if not i in X_fins.columns:
        not_in_the_test_data.append(i)
    else:
        pass
# if ther e are some columns that are not in the testing set, give them a dummy of 0 otherwise the model wont be able to predict
for i in not_in_the_test_data:
    X_fins[i] = 0

print(X_fins.shape,X_fin.shape)
import numpy as np

Pred_test = RF.predict(X_fins)
P = pd.DataFrame(Pred_test.round())# Round off the figures since they are number of tickets boughts and decimals wont make sense

G = pd.concat((test.ride_id,P),1) # Add the IDs to the predictions in the required formats
G.head()
G.to_csv('Baseline_model.csv',index = False) # Save the model to the local machine



(1111, 21) (6249, 27)
(1111, 27) (6249, 20)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
